In [1]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import seaborn as sns
import os
import re

# show all columns
pd.set_option('display.max_columns', None)

In [2]:
# load data rh and data lh subcortical and wm
data_rh = pd.read_excel('aparc_rh_volume_table.xlsx')

data_rh = data_rh.rename(columns ={"rh.aparc.volume" : "ID"})
data_rh['ID'] = data_rh.apply(lambda row: int(row['ID'].lstrip("sub-")), axis = 1)

data_lh = pd.read_excel('aparc_lh_volume_table.xlsx').rename(columns ={"lh.aparc.volume" : "ID"})
data_lh['ID'] = data_lh.apply(lambda row: int(row['ID'].lstrip("sub-")), axis = 1)

sub = pd.read_excel('subcort_vol_DK.xlsx').rename(columns ={"Measure:volume" : "ID"})
sub['ID'] = sub.apply(lambda row: int(row['ID'].lstrip("sub-")), axis = 1)

wm = pd.read_excel('wm_vol_table.xlsx').rename(columns ={"Measure:volume" : "ID"})
wm['ID'] = wm.apply(lambda row: int(row['ID'].lstrip("sub-")), axis = 1)

In [3]:
wm_vol = wm[['ID', 'wm-lh-bankssts', 'wm-lh-caudalanteriorcingulate',
       'wm-lh-caudalmiddlefrontal', 'wm-lh-cuneus', 'wm-lh-entorhinal',
       'wm-lh-fusiform', 'wm-lh-inferiorparietal', 'wm-lh-inferiortemporal',
       'wm-lh-isthmuscingulate', 'wm-lh-lateraloccipital',
       'wm-lh-lateralorbitofrontal', 'wm-lh-lingual',
       'wm-lh-medialorbitofrontal', 'wm-lh-middletemporal',
       'wm-lh-parahippocampal', 'wm-lh-paracentral', 'wm-lh-parsopercularis',
       'wm-lh-parsorbitalis', 'wm-lh-parstriangularis', 'wm-lh-pericalcarine',
       'wm-lh-postcentral', 'wm-lh-posteriorcingulate', 'wm-lh-precentral',
       'wm-lh-precuneus', 'wm-lh-rostralanteriorcingulate',
       'wm-lh-rostralmiddlefrontal', 'wm-lh-superiorfrontal',
       'wm-lh-superiorparietal', 'wm-lh-superiortemporal',
       'wm-lh-supramarginal', 'wm-lh-frontalpole', 'wm-lh-temporalpole',
       'wm-lh-transversetemporal', 'wm-lh-insula', 'wm-rh-bankssts',
       'wm-rh-caudalanteriorcingulate', 'wm-rh-caudalmiddlefrontal',
       'wm-rh-cuneus', 'wm-rh-entorhinal', 'wm-rh-fusiform',
       'wm-rh-inferiorparietal', 'wm-rh-inferiortemporal',
       'wm-rh-isthmuscingulate', 'wm-rh-lateraloccipital',
       'wm-rh-lateralorbitofrontal', 'wm-rh-lingual',
       'wm-rh-medialorbitofrontal', 'wm-rh-middletemporal',
       'wm-rh-parahippocampal', 'wm-rh-paracentral', 'wm-rh-parsopercularis',
       'wm-rh-parsorbitalis', 'wm-rh-parstriangularis', 'wm-rh-pericalcarine',
       'wm-rh-postcentral', 'wm-rh-posteriorcingulate', 'wm-rh-precentral',
       'wm-rh-precuneus', 'wm-rh-rostralanteriorcingulate',
       'wm-rh-rostralmiddlefrontal', 'wm-rh-superiorfrontal',
       'wm-rh-superiorparietal', 'wm-rh-superiortemporal',
       'wm-rh-supramarginal', 'wm-rh-frontalpole', 'wm-rh-temporalpole',
       'wm-rh-transversetemporal', 'wm-rh-insula']]

In [4]:
sub_vol = sub[['ID', 
       'Left-Thalamus-Proper', 'Left-Caudate', 'Left-Putamen', 'Left-Pallidum',
       'Left-Hippocampus',
       'Left-Amygdala', 'Left-Accumbens-area', 'Left-VentralDC',
       'Right-Thalamus-Proper', 'Right-Caudate',
       'Right-Putamen', 'Right-Pallidum', 'Right-Hippocampus',
       'Right-Amygdala', 'Right-Accumbens-area', 'Right-VentralDC',
       
       'CC_Posterior', 'CC_Mid_Posterior', 'CC_Central',
       'CC_Mid_Anterior', 'CC_Anterior', 
       'SupraTentorialVol', 'SupraTentorialVolNotVent',
       ]]

In [5]:
data_rh_vol = data_rh[['ID', 'rh_bankssts_volume', 'rh_caudalanteriorcingulate_volume',
       'rh_caudalmiddlefrontal_volume', 'rh_cuneus_volume',
       'rh_entorhinal_volume', 'rh_fusiform_volume',
       'rh_inferiorparietal_volume', 'rh_inferiortemporal_volume',
       'rh_isthmuscingulate_volume', 'rh_lateraloccipital_volume',
       'rh_lateralorbitofrontal_volume', 'rh_lingual_volume',
       'rh_medialorbitofrontal_volume', 'rh_middletemporal_volume',
       'rh_parahippocampal_volume', 'rh_paracentral_volume',
       'rh_parsopercularis_volume', 'rh_parsorbitalis_volume',
       'rh_parstriangularis_volume', 'rh_pericalcarine_volume',
       'rh_postcentral_volume', 'rh_posteriorcingulate_volume',
       'rh_precentral_volume', 'rh_precuneus_volume',
       'rh_rostralanteriorcingulate_volume', 'rh_rostralmiddlefrontal_volume',
       'rh_superiorfrontal_volume', 'rh_superiorparietal_volume',
       'rh_superiortemporal_volume', 'rh_supramarginal_volume',
       'rh_frontalpole_volume', 'rh_temporalpole_volume',
       'rh_transversetemporal_volume', 'rh_insula_volume']]

In [6]:
data_lh_vol = data_lh[['ID', 'lh_bankssts_volume', 'lh_caudalanteriorcingulate_volume',
       'lh_caudalmiddlefrontal_volume', 'lh_cuneus_volume',
       'lh_entorhinal_volume', 'lh_fusiform_volume',
       'lh_inferiorparietal_volume', 'lh_inferiortemporal_volume',
       'lh_isthmuscingulate_volume', 'lh_lateraloccipital_volume',
       'lh_lateralorbitofrontal_volume', 'lh_lingual_volume',
       'lh_medialorbitofrontal_volume', 'lh_middletemporal_volume',
       'lh_parahippocampal_volume', 'lh_paracentral_volume',
       'lh_parsopercularis_volume', 'lh_parsorbitalis_volume',
       'lh_parstriangularis_volume', 'lh_pericalcarine_volume',
       'lh_postcentral_volume', 'lh_posteriorcingulate_volume',
       'lh_precentral_volume', 'lh_precuneus_volume',
       'lh_rostralanteriorcingulate_volume', 'lh_rostralmiddlefrontal_volume',
       'lh_superiorfrontal_volume', 'lh_superiorparietal_volume',
       'lh_superiortemporal_volume', 'lh_supramarginal_volume',
       'lh_frontalpole_volume', 'lh_temporalpole_volume',
       'lh_transversetemporal_volume', 'lh_insula_volume']]

In [7]:
all_data = ((data_lh_vol.merge(data_rh_vol)).merge(wm_vol)).merge(sub_vol)
all_data

,ID,lh_bankssts_volume,lh_caudalanteriorcingulate_volume,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,lh_entorhinal_volume,lh_fusiform_volume,lh_inferiorparietal_volume,lh_inferiortemporal_volume,lh_isthmuscingulate_volume,lh_lateraloccipital_volume,lh_lateralorbitofrontal_volume,lh_lingual_volume,lh_medialorbitofrontal_volume,lh_middletemporal_volume,lh_parahippocampal_volume,lh_paracentral_volume,lh_parsopercularis_volume,lh_parsorbitalis_volume,lh_parstriangularis_volume,lh_pericalcarine_volume,lh_postcentral_volume,lh_posteriorcingulate_volume,lh_precentral_volume,lh_precuneus_volume,lh_rostralanteriorcingulate_volume,lh_rostralmiddlefrontal_volume,lh_superiorfrontal_volume,lh_superiorparietal_volume,lh_superiortemporal_volume,lh_supramarginal_volume,lh_frontalpole_volume,lh_temporalpole_volume,lh_transversetemporal_volume,lh_insula_volume,rh_bankssts_volume,rh_caudalanteriorcingulate_volume,rh_caudalmiddlefrontal_volume,rh_cuneus_volume,rh_entorhinal_volume,rh_fusiform_volume,rh_inferiorparietal_volume,rh_inferiortemporal_volume,rh_isthmuscingulate_volume,rh_lateraloccipital_volume,rh_lateralorbitofrontal_volume,rh_lingual_volume,rh_medialorbitofrontal_volume,rh_middletemporal_volume,rh_parahippocampal_volume,rh_paracentral_volume,rh_parsopercularis_volume,rh_parsorbitalis_volume,rh_parstriangularis_volume,rh_pericalcarine_volume,rh_postcentral_volume,rh_posteriorcingulate_volume,rh_precentral_volume,rh_precuneus_volume,rh_rostralanteriorcingulate_volume,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume,wm-lh-bankssts,wm-lh-caudalanteriorcingulate,wm-lh-caudalmiddlefrontal,wm-lh-cuneus,wm-lh-entorhinal,wm-lh-fusiform,wm-lh-inferiorparietal,wm-lh-inferiortemporal,wm-lh-isthmuscingulate,wm-lh-lateraloccipital,wm-lh-lateralorbitofrontal,wm-lh-lingual,wm-lh-medialorbitofrontal,wm-lh-middletemporal,wm-lh-parahippocampal,wm-lh-paracentral,wm-lh-parsopercularis,wm-lh-parsorbitalis,wm-lh-parstriangularis,wm-lh-pericalcarine,wm-lh-postcentral,wm-lh-posteriorcingulate,wm-lh-precentral,wm-lh-precuneus,wm-lh-rostralanteriorcingulate,wm-lh-rostralmiddlefrontal,wm-lh-superiorfrontal,wm-lh-superiorparietal,wm-lh-superiortemporal,wm-lh-supramarginal,wm-lh-frontalpole,wm-lh-temporalpole,wm-lh-transversetemporal,wm-lh-insula,wm-rh-bankssts,wm-rh-caudalanteriorcingulate,wm-rh-caudalmiddlefrontal,wm-rh-cuneus,wm-rh-entorhinal,wm-rh-fusiform,wm-rh-inferiorparietal,wm-rh-inferiortemporal,wm-rh-isthmuscingulate,wm-rh-lateraloccipital,wm-rh-lateralorbitofrontal,wm-rh-lingual,wm-rh-medialorbitofrontal,wm-rh-middletemporal,wm-rh-parahippocampal,wm-rh-paracentral,wm-rh-parsopercularis,wm-rh-parsorbitalis,wm-rh-parstriangularis,wm-rh-pericalcarine,wm-rh-postcentral,wm-rh-posteriorcingulate,wm-rh-precentral,wm-rh-precuneus,wm-rh-rostralanteriorcingulate,wm-rh-rostralmiddlefrontal,wm-rh-superiorfrontal,wm-rh-superiorparietal,wm-rh-superiortemporal,wm-rh-supramarginal,wm-rh-frontalpole,wm-rh-temporalpole,wm-rh-transversetemporal,wm-rh-insula,Left-Thalamus-Proper,Left-Caudate,Left-Putamen,Left-Pallidum,Left-Hippocampus,Left-Amygdala,Left-Accumbens-area,Left-VentralDC,Right-Thalamus-Proper,Right-Caudate,Right-Putamen,Right-Pallidum,Right-Hippocampus,Right-Amygdala,Right-Accumbens-area,Right-VentralDC,CC_Posterior,CC_Mid_Posterior,CC_Central,CC_Mid_Anterior,CC_Anterior,SupraTentorialVol,SupraTentorialVolNotVent
0,1,2561,1949,7029,2932,2044,8947,13081,9822,2649,10502,7513,6383,4566,12555,2378,3776,4311,2031,2945,2140,9899,3509,14849,9071,2507,15487,23511,10783,11978,11947,918,1694,1091,7468,2168,1901,6785,2627,1838,8104,14478,9058,2544,11268,7411,7099,4873,14063,2029,4149,3331,3166,4312,2240,9822,3404,14797,9586,1479,15212,21406,11277,11984,10357,929,1808,1161,7000,3198.6,2572.4,6896.9,1753.4,800.2,6196.9,10090.1,4907.1,3919.2,6783.1,6517.2,3919.2,3226.4,5326.5,1471.6,3955.3,3025.2

In [8]:
# return all keys
for key in all_data.keys():
    print("\'"+ key + "\'", end = ',')

'ID','lh_bankssts_volume','lh_caudalanteriorcingulate_volume','lh_caudalmiddlefrontal_volume','lh_cuneus_volume','lh_entorhinal_volume','lh_fusiform_volume','lh_inferiorparietal_volume','lh_inferiortemporal_volume','lh_isthmuscingulate_volume','lh_lateraloccipital_volume','lh_lateralorbitofrontal_volume','lh_lingual_volume','lh_medialorbitofrontal_volume','lh_middletemporal_volume','lh_parahippocampal_volume','lh_paracentral_volume','lh_parsopercularis_volume','lh_parsorbitalis_volume','lh_parstriangularis_volume','lh_pericalcarine_volume','lh_postcentral_volume','lh_posteriorcingulate_volume','lh_precentral_volume','lh_precuneus_volume','lh_rostralanteriorcingulate_volume','lh_rostralmiddlefrontal_volume','lh_superiorfrontal_volume','lh_superiorparietal_volume','lh_superiortemporal_volume','lh_supramarginal_volume','lh_frontalpole_volume','lh_temporalpole_volume','lh_transversetemporal_volume','lh_insula_volume','rh_bankssts_volume','rh_caudalanteriorcingulate_volume','rh_caudalmiddle

In [9]:
# normalize brain data on SupraTentorialVolNotVent
all_data[['lh_bankssts_volume','lh_caudalanteriorcingulate_volume','lh_caudalmiddlefrontal_volume','lh_cuneus_volume','lh_entorhinal_volume','lh_fusiform_volume','lh_inferiorparietal_volume','lh_inferiortemporal_volume','lh_isthmuscingulate_volume','lh_lateraloccipital_volume','lh_lateralorbitofrontal_volume','lh_lingual_volume','lh_medialorbitofrontal_volume','lh_middletemporal_volume','lh_parahippocampal_volume','lh_paracentral_volume','lh_parsopercularis_volume','lh_parsorbitalis_volume','lh_parstriangularis_volume','lh_pericalcarine_volume','lh_postcentral_volume','lh_posteriorcingulate_volume','lh_precentral_volume','lh_precuneus_volume','lh_rostralanteriorcingulate_volume','lh_rostralmiddlefrontal_volume','lh_superiorfrontal_volume','lh_superiorparietal_volume','lh_superiortemporal_volume','lh_supramarginal_volume','lh_frontalpole_volume','lh_temporalpole_volume','lh_transversetemporal_volume','lh_insula_volume','rh_bankssts_volume','rh_caudalanteriorcingulate_volume','rh_caudalmiddlefrontal_volume','rh_cuneus_volume','rh_entorhinal_volume','rh_fusiform_volume','rh_inferiorparietal_volume','rh_inferiortemporal_volume','rh_isthmuscingulate_volume','rh_lateraloccipital_volume','rh_lateralorbitofrontal_volume','rh_lingual_volume','rh_medialorbitofrontal_volume','rh_middletemporal_volume','rh_parahippocampal_volume','rh_paracentral_volume','rh_parsopercularis_volume','rh_parsorbitalis_volume','rh_parstriangularis_volume','rh_pericalcarine_volume','rh_postcentral_volume','rh_posteriorcingulate_volume','rh_precentral_volume','rh_precuneus_volume','rh_rostralanteriorcingulate_volume','rh_rostralmiddlefrontal_volume','rh_superiorfrontal_volume','rh_superiorparietal_volume','rh_superiortemporal_volume','rh_supramarginal_volume','rh_frontalpole_volume','rh_temporalpole_volume','rh_transversetemporal_volume','rh_insula_volume','wm-lh-bankssts','wm-lh-caudalanteriorcingulate','wm-lh-caudalmiddlefrontal','wm-lh-cuneus','wm-lh-entorhinal','wm-lh-fusiform','wm-lh-inferiorparietal','wm-lh-inferiortemporal','wm-lh-isthmuscingulate','wm-lh-lateraloccipital','wm-lh-lateralorbitofrontal','wm-lh-lingual','wm-lh-medialorbitofrontal','wm-lh-middletemporal','wm-lh-parahippocampal','wm-lh-paracentral','wm-lh-parsopercularis','wm-lh-parsorbitalis','wm-lh-parstriangularis','wm-lh-pericalcarine','wm-lh-postcentral','wm-lh-posteriorcingulate','wm-lh-precentral','wm-lh-precuneus','wm-lh-rostralanteriorcingulate','wm-lh-rostralmiddlefrontal','wm-lh-superiorfrontal','wm-lh-superiorparietal','wm-lh-superiortemporal','wm-lh-supramarginal','wm-lh-frontalpole','wm-lh-temporalpole','wm-lh-transversetemporal','wm-lh-insula','wm-rh-bankssts','wm-rh-caudalanteriorcingulate','wm-rh-caudalmiddlefrontal','wm-rh-cuneus','wm-rh-entorhinal','wm-rh-fusiform','wm-rh-inferiorparietal','wm-rh-inferiortemporal','wm-rh-isthmuscingulate','wm-rh-lateraloccipital','wm-rh-lateralorbitofrontal','wm-rh-lingual','wm-rh-medialorbitofrontal','wm-rh-middletemporal','wm-rh-parahippocampal','wm-rh-paracentral','wm-rh-parsopercularis','wm-rh-parsorbitalis','wm-rh-parstriangularis','wm-rh-pericalcarine','wm-rh-postcentral','wm-rh-posteriorcingulate','wm-rh-precentral','wm-rh-precuneus','wm-rh-rostralanteriorcingulate','wm-rh-rostralmiddlefrontal','wm-rh-superiorfrontal','wm-rh-superiorparietal','wm-rh-superiortemporal','wm-rh-supramarginal','wm-rh-frontalpole','wm-rh-temporalpole','wm-rh-transversetemporal','wm-rh-insula','Left-Thalamus-Proper','Left-Caudate','Left-Putamen','Left-Pallidum','Left-Hippocampus','Left-Amygdala','Left-Accumbens-area','Left-VentralDC','Right-Thalamus-Proper','Right-Caudate','Right-Putamen','Right-Pallidum','Right-Hippocampus','Right-Amygdala','Right-Accumbens-area','Right-VentralDC','CC_Posterior','CC_Mid_Posterior','CC_Central','CC_Mid_Anterior','CC_Anterior']] = all_data[['lh_bankssts_volume','lh_caudalanteriorcingulate_volume','lh_caudalmiddlefrontal_volume','lh_cuneus_volume','lh_entorhinal_volume','lh_fusiform_volume','lh_inferiorparietal_volume','lh_inferiortemporal_volume','lh_isthmuscingulate_volume','lh_lateraloccipital_volume','lh_lateralorbitofrontal_volume','lh_lingual_volume','lh_medialorbitofrontal_volume','lh_middletemporal_volume','lh_parahippocampal_volume','lh_paracentral_volume','lh_parsopercularis_volume','lh_parsorbitalis_volume','lh_parstriangularis_volume','lh_pericalcarine_volume','lh_postcentral_volume','lh_posteriorcingulate_volume','lh_precentral_volume','lh_precuneus_volume','lh_rostralanteriorcingulate_volume','lh_rostralmiddlefrontal_volume','lh_superiorfrontal_volume','lh_superiorparietal_volume','lh_superiortemporal_volume','lh_supramarginal_volume','lh_frontalpole_volume','lh_temporalpole_volume','lh_transversetemporal_volume','lh_insula_volume','rh_bankssts_volume','rh_caudalanteriorcingulate_volume','rh_caudalmiddlefrontal_volume','rh_cuneus_volume','rh_entorhinal_volume','rh_fusiform_volume','rh_inferiorparietal_volume','rh_inferiortemporal_volume','rh_isthmuscingulate_volume','rh_lateraloccipital_volume','rh_lateralorbitofrontal_volume','rh_lingual_volume','rh_medialorbitofrontal_volume','rh_middletemporal_volume','rh_parahippocampal_volume','rh_paracentral_volume','rh_parsopercularis_volume','rh_parsorbitalis_volume','rh_parstriangularis_volume','rh_pericalcarine_volume','rh_postcentral_volume','rh_posteriorcingulate_volume','rh_precentral_volume','rh_precuneus_volume','rh_rostralanteriorcingulate_volume','rh_rostralmiddlefrontal_volume','rh_superiorfrontal_volume','rh_superiorparietal_volume','rh_superiortemporal_volume','rh_supramarginal_volume','rh_frontalpole_volume','rh_temporalpole_volume','rh_transversetemporal_volume','rh_insula_volume','wm-lh-bankssts','wm-lh-caudalanteriorcingulate','wm-lh-caudalmiddlefrontal','wm-lh-cuneus','wm-lh-entorhinal','wm-lh-fusiform','wm-lh-inferiorparietal','wm-lh-inferiortemporal','wm-lh-isthmuscingulate','wm-lh-lateraloccipital','wm-lh-lateralorbitofrontal','wm-lh-lingual','wm-lh-medialorbitofrontal','wm-lh-middletemporal','wm-lh-parahippocampal','wm-lh-paracentral','wm-lh-parsopercularis','wm-lh-parsorbitalis','wm-lh-parstriangularis','wm-lh-pericalcarine','wm-lh-postcentral','wm-lh-posteriorcingulate','wm-lh-precentral','wm-lh-precuneus','wm-lh-rostralanteriorcingulate','wm-lh-rostralmiddlefrontal','wm-lh-superiorfrontal','wm-lh-superiorparietal','wm-lh-superiortemporal','wm-lh-supramarginal','wm-lh-frontalpole','wm-lh-temporalpole','wm-lh-transversetemporal','wm-lh-insula','wm-rh-bankssts','wm-rh-caudalanteriorcingulate','wm-rh-caudalmiddlefrontal','wm-rh-cuneus','wm-rh-entorhinal','wm-rh-fusiform','wm-rh-inferiorparietal','wm-rh-inferiortemporal','wm-rh-isthmuscingulate','wm-rh-lateraloccipital','wm-rh-lateralorbitofrontal','wm-rh-lingual','wm-rh-medialorbitofrontal','wm-rh-middletemporal','wm-rh-parahippocampal','wm-rh-paracentral','wm-rh-parsopercularis','wm-rh-parsorbitalis','wm-rh-parstriangularis','wm-rh-pericalcarine','wm-rh-postcentral','wm-rh-posteriorcingulate','wm-rh-precentral','wm-rh-precuneus','wm-rh-rostralanteriorcingulate','wm-rh-rostralmiddlefrontal','wm-rh-superiorfrontal','wm-rh-superiorparietal','wm-rh-superiortemporal','wm-rh-supramarginal','wm-rh-frontalpole','wm-rh-temporalpole','wm-rh-transversetemporal','wm-rh-insula','Left-Thalamus-Proper','Left-Caudate','Left-Putamen','Left-Pallidum','Left-Hippocampus','Left-Amygdala','Left-Accumbens-area','Left-VentralDC','Right-Thalamus-Proper','Right-Caudate','Right-Putamen','Right-Pallidum','Right-Hippocampus','Right-Amygdala','Right-Accumbens-area','Right-VentralDC','CC_Posterior','CC_Mid_Posterior','CC_Central','CC_Mid_Anterior','CC_Anterior']].div(all_data.SupraTentorialVolNotVent, axis = 0)
   
all_data

,ID,lh_bankssts_volume,lh_caudalanteriorcingulate_volume,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,lh_entorhinal_volume,lh_fusiform_volume,lh_inferiorparietal_volume,lh_inferiortemporal_volume,lh_isthmuscingulate_volume,lh_lateraloccipital_volume,lh_lateralorbitofrontal_volume,lh_lingual_volume,lh_medialorbitofrontal_volume,lh_middletemporal_volume,lh_parahippocampal_volume,lh_paracentral_volume,lh_parsopercularis_volume,lh_parsorbitalis_volume,lh_parstriangularis_volume,lh_pericalcarine_volume,lh_postcentral_volume,lh_posteriorcingulate_volume,lh_precentral_volume,lh_precuneus_volume,lh_rostralanteriorcingulate_volume,lh_rostralmiddlefrontal_volume,lh_superiorfrontal_volume,lh_superiorparietal_volume,lh_superiortemporal_volume,lh_supramarginal_volume,lh_frontalpole_volume,lh_temporalpole_volume,lh_transversetemporal_volume,lh_insula_volume,rh_bankssts_volume,rh_caudalanteriorcingulate_volume,rh_caudalmiddlefrontal_volume,rh_cuneus_volume,rh_entorhinal_volume,rh_fusiform_volume,rh_inferiorparietal_volume,rh_inferiortemporal_volume,rh_isthmuscingulate_volume,rh_lateraloccipital_volume,rh_lateralorbitofrontal_volume,rh_lingual_volume,rh_medialorbitofrontal_volume,rh_middletemporal_volume,rh_parahippocampal_volume,rh_paracentral_volume,rh_parsopercularis_volume,rh_parsorbitalis_volume,rh_parstriangularis_volume,rh_pericalcarine_volume,rh_postcentral_volume,rh_posteriorcingulate_volume,rh_precentral_volume,rh_precuneus_volume,rh_rostralanteriorcingulate_volume,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume,wm-lh-bankssts,wm-lh-caudalanteriorcingulate,wm-lh-caudalmiddlefrontal,wm-lh-cuneus,wm-lh-entorhinal,wm-lh-fusiform,wm-lh-inferiorparietal,wm-lh-inferiortemporal,wm-lh-isthmuscingulate,wm-lh-lateraloccipital,wm-lh-lateralorbitofrontal,wm-lh-lingual,wm-lh-medialorbitofrontal,wm-lh-middletemporal,wm-lh-parahippocampal,wm-lh-paracentral,wm-lh-parsopercularis,wm-lh-parsorbitalis,wm-lh-parstriangularis,wm-lh-pericalcarine,wm-lh-postcentral,wm-lh-posteriorcingulate,wm-lh-precentral,wm-lh-precuneus,wm-lh-rostralanteriorcingulate,wm-lh-rostralmiddlefrontal,wm-lh-superiorfrontal,wm-lh-superiorparietal,wm-lh-superiortemporal,wm-lh-supramarginal,wm-lh-frontalpole,wm-lh-temporalpole,wm-lh-transversetemporal,wm-lh-insula,wm-rh-bankssts,wm-rh-caudalanteriorcingulate,wm-rh-caudalmiddlefrontal,wm-rh-cuneus,wm-rh-entorhinal,wm-rh-fusiform,wm-rh-inferiorparietal,wm-rh-inferiortemporal,wm-rh-isthmuscingulate,wm-rh-lateraloccipital,wm-rh-lateralorbitofrontal,wm-rh-lingual,wm-rh-medialorbitofrontal,wm-rh-middletemporal,wm-rh-parahippocampal,wm-rh-paracentral,wm-rh-parsopercularis,wm-rh-parsorbitalis,wm-rh-parstriangularis,wm-rh-pericalcarine,wm-rh-postcentral,wm-rh-posteriorcingulate,wm-rh-precentral,wm-rh-precuneus,wm-rh-rostralanteriorcingulate,wm-rh-rostralmiddlefrontal,wm-rh-superiorfrontal,wm-rh-superiorparietal,wm-rh-superiortemporal,wm-rh-supramarginal,wm-rh-frontalpole,wm-rh-temporalpole,wm-rh-transversetemporal,wm-rh-insula,Left-Thalamus-Proper,Left-Caudate,Left-Putamen,Left-Pallidum,Left-Hippocampus,Left-Amygdala,Left-Accumbens-area,Left-VentralDC,Right-Thalamus-Proper,Right-Caudate,Right-Putamen,Right-Pallidum,Right-Hippocampus,Right-Amygdala,Right-Accumbens-area,Right-VentralDC,CC_Posterior,CC_Mid_Posterior,CC_Central,CC_Mid_Anterior,CC_Anterior,SupraTentorialVol,SupraTentorialVolNotVent
0,1,0.002964,0.002256,0.008136,0.003394,0.002366,0.010357,0.015142,0.011369,0.003066,0.012157,0.008697,0.007389,0.005285,0.014533,0.002753,0.004371,0.004990,0.002351,0.003409,0.002477,0.011459,0.004062,0.017188,0.010500,0.002902,0.017927,0.027215,0.012482,0.013865,0.013829,0.001063,0.001961,0.001263,0.008645,0.002510,0.002200,0.007854,0.003041,0.002128,0.009381,0.016759,0.010485,0.002945,0.013043,0.008579,0.008217,0.005641,0.016279,0.002349,0.004803,0.003856,0.003665,0.0049

In [10]:
# save data to excel
all_data.to_excel('norm_SupraTentorialVolNotVent_DK_WM_vol.xlsx')

In [40]:
# data = all_data.drop(columns = ['SupraTentorialVol', 'SupraTentorialVolNotVent'])
# data

,ID,lh_bankssts_volume,lh_caudalanteriorcingulate_volume,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,lh_entorhinal_volume,lh_fusiform_volume,lh_inferiorparietal_volume,lh_inferiortemporal_volume,lh_isthmuscingulate_volume,lh_lateraloccipital_volume,lh_lateralorbitofrontal_volume,lh_lingual_volume,lh_medialorbitofrontal_volume,lh_middletemporal_volume,lh_parahippocampal_volume,lh_paracentral_volume,lh_parsopercularis_volume,lh_parsorbitalis_volume,lh_parstriangularis_volume,lh_pericalcarine_volume,lh_postcentral_volume,lh_posteriorcingulate_volume,lh_precentral_volume,lh_precuneus_volume,lh_rostralanteriorcingulate_volume,lh_rostralmiddlefrontal_volume,lh_superiorfrontal_volume,lh_superiorparietal_volume,lh_superiortemporal_volume,lh_supramarginal_volume,lh_frontalpole_volume,lh_temporalpole_volume,lh_transversetemporal_volume,lh_insula_volume,rh_bankssts_volume,rh_caudalanteriorcingulate_volume,rh_caudalmiddlefrontal_volume,rh_cuneus_volume,rh_entorhinal_volume,rh_fusiform_volume,rh_inferiorparietal_volume,rh_inferiortemporal_volume,rh_isthmuscingulate_volume,rh_lateraloccipital_volume,rh_lateralorbitofrontal_volume,rh_lingual_volume,rh_medialorbitofrontal_volume,rh_middletemporal_volume,rh_parahippocampal_volume,rh_paracentral_volume,rh_parsopercularis_volume,rh_parsorbitalis_volume,rh_parstriangularis_volume,rh_pericalcarine_volume,rh_postcentral_volume,rh_posteriorcingulate_volume,rh_precentral_volume,rh_precuneus_volume,rh_rostralanteriorcingulate_volume,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume,wm-lh-bankssts,wm-lh-caudalanteriorcingulate,wm-lh-caudalmiddlefrontal,wm-lh-cuneus,wm-lh-entorhinal,wm-lh-fusiform,wm-lh-inferiorparietal,wm-lh-inferiortemporal,wm-lh-isthmuscingulate,wm-lh-lateraloccipital,wm-lh-lateralorbitofrontal,wm-lh-lingual,wm-lh-medialorbitofrontal,wm-lh-middletemporal,wm-lh-parahippocampal,wm-lh-paracentral,wm-lh-parsopercularis,wm-lh-parsorbitalis,wm-lh-parstriangularis,wm-lh-pericalcarine,wm-lh-postcentral,wm-lh-posteriorcingulate,wm-lh-precentral,wm-lh-precuneus,wm-lh-rostralanteriorcingulate,wm-lh-rostralmiddlefrontal,wm-lh-superiorfrontal,wm-lh-superiorparietal,wm-lh-superiortemporal,wm-lh-supramarginal,wm-lh-frontalpole,wm-lh-temporalpole,wm-lh-transversetemporal,wm-lh-insula,wm-rh-bankssts,wm-rh-caudalanteriorcingulate,wm-rh-caudalmiddlefrontal,wm-rh-cuneus,wm-rh-entorhinal,wm-rh-fusiform,wm-rh-inferiorparietal,wm-rh-inferiortemporal,wm-rh-isthmuscingulate,wm-rh-lateraloccipital,wm-rh-lateralorbitofrontal,wm-rh-lingual,wm-rh-medialorbitofrontal,wm-rh-middletemporal,wm-rh-parahippocampal,wm-rh-paracentral,wm-rh-parsopercularis,wm-rh-parsorbitalis,wm-rh-parstriangularis,wm-rh-pericalcarine,wm-rh-postcentral,wm-rh-posteriorcingulate,wm-rh-precentral,wm-rh-precuneus,wm-rh-rostralanteriorcingulate,wm-rh-rostralmiddlefrontal,wm-rh-superiorfrontal,wm-rh-superiorparietal,wm-rh-superiortemporal,wm-rh-supramarginal,wm-rh-frontalpole,wm-rh-temporalpole,wm-rh-transversetemporal,wm-rh-insula,Left-Thalamus-Proper,Left-Caudate,Left-Putamen,Left-Pallidum,Left-Hippocampus,Left-Amygdala,Left-Accumbens-area,Left-VentralDC,Right-Thalamus-Proper,Right-Caudate,Right-Putamen,Right-Pallidum,Right-Hippocampus,Right-Amygdala,Right-Accumbens-area,Right-VentralDC,CC_Posterior,CC_Mid_Posterior,CC_Central,CC_Mid_Anterior,CC_Anterior
0,1,0.002766,0.002105,0.007592,0.003167,0.002208,0.009663,0.014128,0.010609,0.002861,0.011343,0.008115,0.006894,0.004932,0.013560,0.002568,0.004078,0.004656,0.002194,0.003181,0.002311,0.010692,0.003790,0.016038,0.009797,0.002708,0.016727,0.025394,0.011646,0.012937,0.012904,0.000992,0.001830,0.001178,0.008066,0.002342,0.002053,0.007328,0.002837,0.001985,0.008753,0.015637,0.009783,0.002748,0.012170,0.008004,0.007667,0.005263,0.015189,0.002191,0.004481,0.003598,0.003420,0.004657,0.002419,0.010609,0.003677,0.015982,0.01